In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from joblib import load
import pickle
from pathlib import Path
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
class CentralMontneyRatePred:
    def __init__(self, static_inputs, static_input_keys, prod_hist=None, shut_list=None):
        """
        Parameters:
        static_inputs (dict): A dictionary containing the static inputs provided by the user.
        prod_hist (array): A 2D numpy array representing historical production data with shape (n_timesteps, 3).
        shut_list (list): A list of indices representing shut-in periods (optional).
        """
        # Define the keys for peak input
        self.peak_input_keys = static_input_keys
        self.gbr_model_p = self._load_gbr_model()
        self.AI_model = self.load_model()
        self.static_inputs = self.convert_static_inputs(static_inputs)
        self.prod_hist = self._pad_prod_hist(prod_hist)  # Pad or initialize prod_hist to (60, 3)
        self.shut_ins = self._convert_shut_list(shut_list)  # Convert or initialize shut_list to (60, 1)
        self.hist_len = len(prod_hist) if prod_hist is not None else 0  # Determine hist_len
        self.dynamic_df = None  # Dynamic inputs generated from static inputs
        self.scalers = self._load_scalers()

        # Predict PeakTime and PeakRate if they are not provided
        if 'PeakTime' not in self.static_inputs or 'PeakRate' not in self.static_inputs:
            self.predict_peak()

    def _pad_prod_hist(self, prod_hist):
        if prod_hist is None:
            return np.zeros((60, 3))  # Default to zeros if no production history is provided

        padded_hist = np.zeros((60, 3))  # Initialize a (60, 3) array with zeros
        length = min(len(prod_hist), 60)  # Determine how much of prod_hist can be used
        padded_hist[:length, :] = prod_hist[:length, :]  # Fill with available production history
        print(padded_hist)
        return padded_hist

    def _convert_shut_list(self, shut_list):
        shut_ins = np.zeros((60, 1))  # Initialize a (60, 1) array with zeros
        if shut_list is not None:
            for index in shut_list:
                if 0 <= index < 60:  # Ensure the index is within bounds
                    shut_ins[index, 0] = 1
        return shut_ins
        
    def _load_gbr_model(self):
        try:
            return load('c:\\Users\\ziming4\\OneDrive - University of Alberta\\Porject_OPI\\WebPrototype0823\\OpenPredictionInterface\\backend\\app/integrations/Montney/multi_gbr_regressor_peak.joblib')
        except Exception as e:
            raise FileNotFoundError("GBR model could not be loaded. Ensure the file path is correct.") from e

    def load_model(self):
        mask_special_val = -1e9
        threshold_active = 0

        def thresholded_tanh(x, threshold=threshold_active):
            thresholds = tf.constant(threshold, dtype=x.dtype)
            thresholds = tf.reshape(thresholds, (1, 1, -1))
            tanh_x = tf.tanh(x)
            adjusted_tanh = (tanh_x + 1) / 2 * (1 - thresholds) + thresholds  # Scales and shifts the output
            return adjusted_tanh

        def masked_loss_function(y_true, y_pred):
            mask = K.cast(K.not_equal(y_true, mask_special_val), K.floatx())
            y_true_masked = y_true * mask
            y_pred_masked = y_pred * mask
            loss = K.mean(K.square(y_pred_masked - y_true_masked), axis=-1)
            return loss

        # Load the model with the registered custom objects
        try:
            model = keras.models.load_model(
                'c:\\Users\\ziming4\\OneDrive - University of Alberta\\Porject_OPI\\WebPrototype0823\\OpenPredictionInterface\\backend\\app/integrations/Montney/MED_DUDS_denoise_1024.keras',
                custom_objects={
                    'masked_loss_function': masked_loss_function,
                    'thresholded_tanh': thresholded_tanh
                }
            )
            return model
        except Exception as e:
            print(f"Error details: {e}")
            raise FileNotFoundError("AI model could not be loaded.") from e

    def _load_scalers(self):
        try:
            scalers = {}
            scaler_paths = ['scaler_encoder.pkl', 'scaler_decoder_D.pkl', 'scaler_decoder_S.pkl', 'scaler_y_pred_3phase.pkl']
            scaler_names = ['scaler_encoder', 'scaler_decoder_D', 'scaler_decoder_S', 'scaler_y_pred']

            for name, path in zip(scaler_names, scaler_paths):
                with open(Path('c:\\Users\\ziming4\\OneDrive - University of Alberta\\Porject_OPI\\WebPrototype0823\\OpenPredictionInterface\\backend\\app/integrations/Montney/ml_model_scalers') / path, 'rb') as file:
                    scalers[name] = pickle.load(file)
            return scalers

        except Exception as e:
            raise FileNotFoundError("One or more scaler files could not be loaded. Ensure the file paths are correct.") from e
        
    def classify_and_update_feature(self, feature_name: str, feature_dict: dict, encoding_dict: dict) -> dict:
        """Classifies and updates a feature using the given encoding dictionary."""
        updated_features = feature_dict.copy()
        feature_value = updated_features.get(feature_name, "").upper()

        if feature_name in encoding_dict:
            if feature_value in encoding_dict[feature_name]:
                updated_features[feature_name] = encoding_dict[feature_name][feature_value]
            else:
                raise ValueError(
                    f"Value '{feature_value}' not found in label encodings for '{feature_name}'. "
                )
        else:
            raise ValueError(f"Feature '{feature_name}' not found in label encodings.")
        return updated_features
    
    def convert_static_inputs(self, static_inputs):
        with open('c:\\Users\\ziming4\\OneDrive - University of Alberta\\Porject_OPI\\WebPrototype0823\\OpenPredictionInterface\\backend\\app/integrations/Montney/label_encodings.pkl', 'rb') as file:
            label_encodings = pickle.load(file)
        feature_names = ['ENVInterval', 'County', 'ENVOperator','WellPadDirection'] 
        for feat_name in feature_names:
            try:
                static_inputs = self.classify_and_update_feature(feat_name, static_inputs, label_encodings)
            except ValueError as e:
                print(f"Warning: {e}")
                
        return static_inputs

    def predict_peak(self):
        peak_inputs = {key: self.static_inputs[key] for key in self.peak_input_keys}
        p_inputs = pd.DataFrame([peak_inputs])
        peaks = self.gbr_model_p.predict(p_inputs)
        self.static_inputs['PeakTime'] = peaks[0][0]
        self.static_inputs['PeakRate'] = peaks[0][1]

    def create_dynamic_df(self):
        start_year = int(self.static_inputs['StartYear'])
        start_month = int(self.static_inputs.get('Month', 1))  # Use January (1) as default if 'Month' is missing
        start_date = datetime(start_year, start_month, 1)
        date_list = [start_date + relativedelta(months=+i) for i in range(60)]

        self.dynamic_df = pd.DataFrame({
            'TotalProdMonths': list(range(1, 61)),
            'Year': [date.year for date in date_list],
            'Month': [date.month for date in date_list],
            'ShutIns': self.shut_ins.flatten()
        })

    def predict_profile(self):
        if not self.static_inputs or self.dynamic_df is None:
            raise ValueError("Static inputs must be set, and dynamic inputs must be generated before prediction.")

        # Prepare dynamic and static inputs
        dynamic_inputs = self.dynamic_df[['TotalProdMonths', 'Year', 'Month', 'ShutIns']].values
        if self.hist_len == 0:
            self.hist_len = 1 

        # Encoder input preparation
        encoder_inputs = np.concatenate((dynamic_inputs, self.prod_hist), axis=1)
        encoder_inputs_scaled = self.scalers['scaler_encoder'].transform(encoder_inputs.reshape(-1, 7)).reshape(1, 60, 7)
        encoder_inputs_scaled[:, self.hist_len:, :] = -1e9  # Apply mask beyond the history length

        # Decoder D inputs preparation
        decoder_D_inputs_scaled = self.scalers['scaler_decoder_D'].transform(dynamic_inputs.reshape(-1, 4)).reshape(1, 60, 4)
        decoder_D_inputs_scaled[:, :self.hist_len, :] = -1e9  # Apply mask up to history length
        
        # Decoder S inputs preparation
        NN_static_inputs = {key: self.static_inputs[key] for key in self.peak_input_keys + ['PeakTime', 'PeakRate']}
        decoder_S_inputs = np.array(list(NN_static_inputs.values()), dtype='float32').reshape(1, -1)
        decoder_S_inputs_scaled = self.scalers['scaler_decoder_S'].transform(decoder_S_inputs)

        # Predict the profile
        y_pred = self.AI_model.predict([encoder_inputs_scaled, decoder_D_inputs_scaled, decoder_S_inputs_scaled])
        y_pred[:, 0, :] = 0  # Change the first timestep as zero
        y_pred_restored = self.scalers['scaler_y_pred'].inverse_transform(y_pred.reshape(-1, 3)).reshape(1, 60, 3)
        y_pred_restored[:, :self.hist_len, :] = self.prod_hist[:self.hist_len, :]
        
        return y_pred_restored

In [5]:
import json

def Montney_pred_MED(static_inputs):
    """
    Production forecasting for Central Montney using a deep learning model.
    - 'static_inputs' (dic): File name. Either user inputs or according to the related chat history.
    - 'prod_hist' (list): Production history data. Defaults to None if not provided or empty list.
    - 'shut_list' (list): Shut-in periods. Defaults to None if not provided or empty list.
    """
    
    # Define required static input keys
    static_input_keys = [
        'Longitude', 'Latitude', 'ENVInterval', 'StartYear', 'County', 'ElevationGL_FT',
        'TotalOrganicCarbon_WTPCT', 'HeightOfHCPV_FT', 'HCPV_PCT', 'PhiH_FT',
        'WaterSaturation_PCT', 'NonClayVolume_PCT', 'ClayVolume_PCT',
        'EffectivePorosity_PCT', 'DensityPorosity_PCT', 'Resistivity_OHMSM',
        'BulkDensity_GPerCC', 'GammaRay_API', 'Isopach_FT', 'SubseaBaseDepth_FT',
        'SubseaTopDepth_FT', 'BottomOfZone_FT', 'TopOfZone_FT', 'GasGravity_SG',
        'ENVOperator', 'WellPadDirection', 'AverageStageSpacing_FT', 'FracStages',
        'CompletionTime_DAYS', 'LateralLength_FT', 'MD_FT', 'TVD_FT'
    ]

    # Check for missing static inputs
    missing_keys = [key for key in static_input_keys if key not in static_inputs or static_inputs[key] is None]
    
    if missing_keys:
        return json.dumps({
            "error": f"Missing required static inputs: {', '.join(missing_keys)}",
            "message for python_repl": "There is an error. Do not generate any plots."
        })

    # Ensure 'Month' key exists in static_inputs, defaulting to 1 if not present
    static_inputs.setdefault('Month', 1)

    # Initialize the model with the given inputs
    model = CentralMontneyRatePred(static_inputs, static_input_keys, prod_hist=None, shut_list=None)

    # Generate the dynamic inputs
    model.create_dynamic_df()

    # Predict the production profile
    profile = model.predict_profile() #[1,60,1]

    # Prepare the final results
    output_results = {}

    output_results['gas_rate']= profile[0,:,0].tolist() #  gas
    output_results['oil_rate']= profile[0,:,1].tolist() #  oil 
    output_results['water_rate']= profile[0,:,2].tolist() # water
    # return json.dumps(output_results)
    return output_results

In [6]:
inputs = {'Longitude': -120.37108, 'Latitude': 55.962475, 'ENVInterval': "LOWER MONTNEY A", 'StartYear': 2015, 'Month': 1,
           'County': "SALT CREEK", 'ElevationGL_FT': 2326.4050715475264, 'TotalOrganicCarbon_WTPCT': 2.201066488726354,
             'HeightOfHCPV_FT': 8.808171558963744, 'HCPV_PCT': 0.04914084647439048, 'PhiH_FT': 10.57039430708461,
               'WaterSaturation_PCT': 0.14695686332702404, 'NonClayVolume_PCT': 0.8893624230966368, 'ClayVolume_PCT': 0.05920733462785699,
                 'EffectivePorosity_PCT': 0.061576428149606774, 'DensityPorosity_PCT': 0.06691177902845037, 'Resistivity_OHMSM': 88.48815046925344,
                   'BulkDensity_GPerCC': 2.5971327125627113, 'GammaRay_API': 130.2878673089267, 'Isopach_FT': 175.2161426462049,
                     'SubseaBaseDepth_FT': -4842.154247107849, 'SubseaTopDepth_FT': -4654.953653646224, 'BottomOfZone_FT': 7041.939095090544,
                       'TopOfZone_FT': 6862.18474457186, 'GasGravity_SG': 0.7021147491786565, 'ENVOperator': "CENOVUS", 'WellPadDirection':"E",
                         'AverageStageSpacing_FT': 500, 'FracStages': 20,'CompletionTime_DAYS': 20,
                           'LateralLength_FT': 1000, 'MD_FT': 10000, 'TVD_FT': 2000}

y_pred = Montney_pred_MED(static_inputs = inputs)
# y_pred = Montney_pred_MED(static_inputs = inputs, shut_list = [15,16,56,57,58])

print(y_pred['gas_rate'])
print(y_pred['oil_rate'])
print(y_pred['water_rate'])

c:\Users\ziming4\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ziming4\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\ziming4\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\ziming4\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinM

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 939ms/step
[0.0, 899.7792358398438, 1201.938720703125, 1282.2681884765625, 1235.5023193359375, 1146.595947265625, 1063.169189453125, 993.8115234375, 936.3875122070312, 888.0675659179688, 846.8330078125, 811.2733154296875, 778.4771728515625, 762.7698364257812, 736.1774291992188, 706.9515380859375, 678.9669189453125, 653.671630859375, 631.2133178710938, 611.3226318359375, 593.6499633789062, 577.8733520507812, 563.7200317382812, 550.9564819335938, 537.2736206054688, 534.3765869140625, 524.1363525390625, 511.3260192871094, 498.3046875, 486.1567077636719, 475.13067626953125, 465.17156982421875, 456.1492004394531, 447.934326171875, 440.4193115234375, 433.50823974609375, 424.813720703125, 424.6185302734375, 418.8636169433594, 411.00909423828125, 402.76837158203125, 394.9989318847656, 387.9043273925781, 381.4578857421875, 375.5753479003906, 370.1707763671875, 365.1707763671875, 360.5252990722656, 353.93206787109375, 354.4033508300781, 350.43438720703125, 344.7820739